In [15]:
import pandas as pd
hikes = pd.read_csv('hikes.csv')
hikes

,Activity Type,Date,Favorite,Title,Distance,Calories,Time,Avg HR,Max HR,Aerobic TE,...,Body Battery Drain,Min Temp,Decompression,Best Lap Time,Number of Laps,Max Temp,Moving Time,Elapsed Time,Min Elevation,Max Elevation
0,Hiking,2026-01-15 08:53:55,False,Amecameca Hiking,5.05,"1,482",02:57:08,136,180,3.2,...,-17,68.0,No,02:57:08,1,84.2,01:56:43,03:27:25,"12,089","13,083"


In [16]:
runs = pd.read_csv('runs.csv')
runs

,Activity Type,Date,Favorite,Title,Distance,Calories,Time,Avg HR,Max HR,Aerobic TE,...,Body Battery Drain,Min Temp,Decompression,Best Lap Time,Number of Laps,Max Temp,Moving Time,Elapsed Time,Min Elevation,Max Elevation
0,Treadmill Running,2026-01-24 10:10:33,False,Treadmill Running,3.04,383,00:31:02,148,166,3.0,...,-10,71.6,No,00:00:24.2,4,82.4,00:30:15,00:31:02,--,--
1,Treadmill Running,2026-01-21 18:40:35,False,Treadmill Running,1.12,140,00:12:07,140,151,2.0,...,-2,80.6,No,00:01:14.8,2,86.0,00:11:52,00:12:07,--,--
2,Running,2026-01-18 17:47:03,False,Running,0.87,135,00:10:04,156,190,2.5,...,--,71.6,No,00:10:04,1,78.8,00:08:55,00:12:57,"7,973","7,980"
3,Running,2026-01-14 09:03:48,False,Cuauhtémoc Running,0.48,54,00:04:54.1,134,145,0.8,...,-1,69.8,No,00:04:54.1,1,71.6,00:04:33,00:04:58.7,"6,520","6,528"
4,Treadmill Running,2026-01-09 19:06:44,False,Treadmill Running,1.52,215,00:18:09,143,162,2.3,...,-3,77.0,No,00:05:40.3,2,84.2,00:17:32,00:18:12,--,--
5,Running,2026-01-07 19:32:34,False,Chicago Running,1.77,230,00:16:54,159,181,2.8,...,-3,69.8,No,00:06:58.8,2,78.8,00:16:31,00:17:16,604,615
6,Running,2026-01-05 18:09:42,False,Chicago Running,2.15,225,00:21:30,135,177,2.4,...,-5,64.4,No,00:01:17.0,3,78.8,00:20:55,00:23:44,611,625
7,Running,2025-12-22 17:30:49,False,South Barrington Running,1.02,124,00:10:02,137,152,2.0,...,-1,73.4,No,00:00:07.8,2,84.2,00:09:51,00:10:03,470,484


In [28]:
import pandas as pd

# -----------------------------
# Load Files
# -----------------------------
runs = pd.read_csv("runs.csv")
hikes = pd.read_csv("hikes.csv")
old_log = pd.read_csv("ultra50kmRuns.csv")  # your first training log CSV

# -----------------------------
# Helper: Clean Garmin Numeric Fields
# -----------------------------
def clean_numeric(series):
    return (
        series.astype(str)
        .str.replace(",", "", regex=False)
        .replace("--", None)
        .astype(float)
    )

# -----------------------------
# Clean RUNS data
# -----------------------------
runs["Distance"] = clean_numeric(runs["Distance"])
runs["Calories"] = clean_numeric(runs["Calories"])
runs["Total Ascent"] = clean_numeric(runs["Total Ascent"])
runs["Total Descent"] = clean_numeric(runs["Total Descent"])
runs["Avg HR"] = clean_numeric(runs["Avg HR"])
runs["Max HR"] = clean_numeric(runs["Max HR"])

# -----------------------------
# Clean HIKES data
# -----------------------------
hikes["Distance"] = clean_numeric(hikes["Distance"])
hikes["Calories"] = clean_numeric(hikes["Calories"])
hikes["Ascent"] = clean_numeric(hikes["Ascent"])
hikes["Total Descent"] = clean_numeric(hikes["Total Descent"])
hikes["Avg HR"] = clean_numeric(hikes["Avg HR"])
hikes["Max HR"] = clean_numeric(hikes["Max HR"])

# -----------------------------
# Convert RUNS → Target Schema
# -----------------------------
runs_out = pd.DataFrame({
    "Date": pd.to_datetime(runs["Date"]).dt.date,
    "Distance": runs["Distance"],
    "TotalTime": runs["Time"],
    "MovingTime": runs["Moving Time"],
    "AveragePace": runs["Avg Pace"],
    "AverageMovingPace": runs["Avg Pace"],
    "ElevationGain": runs["Total Ascent"].fillna(0),
    "ElevationDescent": runs["Total Descent"].fillna(0),
    "TerrainType": runs["Activity Type"].apply(
        lambda x: "Treadmill" if "Treadmill" in x else "Track" if "Track" in x else "Road"
    ),
    "RunType": "Easy",
    "AvgHeartRate": runs["Avg HR"],
    "MaxHeartRate": runs["Max HR"],
    "CaloriesBurnt": runs["Calories"],
    "EstimatedSweatLoss": None,
    "Felt": None,
    "LegFatigue": None,
    "Notes": None
})

# -----------------------------
# Convert HIKES → Target Schema
# -----------------------------
hikes_out = pd.DataFrame({
    "Date": pd.to_datetime(hikes["Date"]).dt.date,
    "Distance": hikes["Distance"],
    "TotalTime": hikes["Time"],
    "MovingTime": hikes["Moving Time"],
    "AveragePace": hikes["Avg Pace"],
    "AverageMovingPace": hikes["Avg Pace"],
    "ElevationGain": hikes["Ascent"],
    "ElevationDescent": hikes["Total Descent"],
    "TerrainType": "Trail",
    "RunType": "Hike",  # <-- as requested
    "AvgHeartRate": hikes["Avg HR"],
    "MaxHeartRate": hikes["Max HR"],
    "CaloriesBurnt": hikes["Calories"],
    "EstimatedSweatLoss": None,
    "Felt": None,
    "LegFatigue": None,
    "Notes": None
})

# -----------------------------
# Combine New Activities
# -----------------------------
combined = pd.concat([runs_out, hikes_out], ignore_index=True)

# -----------------------------
# Bring in Old Notes
# -----------------------------
old_log["Date"] = pd.to_datetime(old_log["Date"]).dt.date

notes_lookup = old_log[["Date", "Distance", "Notes"]]

combined = combined.merge(
    notes_lookup,
    on=["Date", "Distance"],
    how="left",
    suffixes=("", "_old")
)

# Fill Notes from old log where available
combined["Notes"] = combined["Notes_old"].combine_first(combined["Notes"])
combined.drop(columns=["Notes_old"], inplace=True)

# -----------------------------
# Final Sort + Export
# -----------------------------
final_df = combined.sort_values("Date")

final_df.to_csv("log.csv", index=False)

# print("✅ Training log updated and saved as ultra_training_log_updated.csv")
# print(final_df.head())
final_df


,Date,Distance,TotalTime,MovingTime,AveragePace,AverageMovingPace,ElevationGain,ElevationDescent,TerrainType,RunType,AvgHeartRate,MaxHeartRate,CaloriesBurnt,EstimatedSweatLoss,Felt,LegFatigue,Notes
10,2025-12-22,1.02,00:10:02,00:09:51,9:53,9:53,16.0,13.0,Road,Easy,137.0,152.0,124.0,None,None,None,First run of my training for my first 50km rac...
9,2026-01-05,2.15,00:21:30,00:20:55,9:59,9:59,0.0,0.0,Road,Easy,135.0,177.0,225.0,None,None,None,Tried to run for 30 minutes but only did 22 be...
8,2026-01-07,1.77,00:16:54,00:16:31,9:33,9:33,16.0,13.0,Road,Easy,159.0,181.0,230.0,None,None,None,Ran to a restaurant where I hung out with Pric...
7,2026-01-09,1.52,00:18:09,00:17:32,11:56,11:56,0.0,0.0,Treadmill,Easy,143.0,162.0,215.0,None,None,None,Did a full body workout and decided to get a l...
6,2026-01-14,0.48,00:04:54.1,00:04:33,10:10,10:10,0.0,3.0,Road,Easy,134.0,145.0,54.0,None,None,None,Very short run to meet my mom at a Breakfast s...
11,2026-01-15,5.05,02:57:08,01:56:43,35:04,35:04,1083.0,948.0,Trail,Hike,136.0,180.0,1482.0,None,None,None,5mile hike. Was my first ever hike at 3800 met...
5,2026-01-18,0.87,00:10:04,00:08:55,11:31,11:31,3.0,3.0,Road,Easy,156.0,190.0,135.0,None,None,None,Very short run. This was my final run in Mexic...
4,2026-01-21,1.12,00:12:07,00:11:52,10:47,10:47,0.0,0.0,Treadmill,Easy,140.0,151.0,140.0,None,None,None,First official day of training where I will be...
3,2026-01-24,3.04,00:31:02,00:30:15,10:13,10:13,0.0,0.0,Treadmill,Easy,148.0,166.0,383.0,None,None,None,None
2,2026-01-25,1.36,00:13:14,00:13:09,9:42,9:42,3.0,10.0,Road,Easy,163.0,191.0,186.0,None,None,None,None


In [30]:
import pandas as pd
df =pd.read_csv('log.csv')
df.to_json('ultra50km.json', orient='records')
df

,Date,Distance,TotalTime,MovingTime,AveragePace,AverageMovingPace,ElevationGain,ElevationDescent,TerrainType,RunType,AvgHeartRate,MaxHeartRate,CaloriesBurnt,EstimatedSweatLoss,Felt,LegFatigue,Notes
0,2025-12-22,1.02,00:10:02,00:09:51,9:53,9:53,16.0,13.0,Road,Easy,137.0,152.0,124.0,NaN,NaN,NaN,First run of my training for my first 50km rac...
1,2026-01-05,2.15,00:21:30,00:20:55,9:59,9:59,0.0,0.0,Road,Easy,135.0,177.0,225.0,NaN,NaN,NaN,Tried to run for 30 minutes but only did 22 be...
2,2026-01-07,1.77,00:16:54,00:16:31,9:33,9:33,16.0,13.0,Road,Easy,159.0,181.0,230.0,NaN,NaN,NaN,Ran to a restaurant where I hung out with Pric...
3,2026-01-09,1.52,00:18:09,00:17:32,11:56,11:56,0.0,0.0,Treadmill,Easy,143.0,162.0,215.0,NaN,NaN,NaN,Did a full body workout and decided to get a l...
4,2026-01-14,0.48,00:04:54.1,00:04:33,10:10,10:10,0.0,3.0,Road,Easy,134.0,145.0,54.0,NaN,NaN,NaN,Very short run to meet my mom at a Breakfast s...
5,2026-01-15,5.05,02:57:08,01:56:43,35:04,35:04,1083.0,948.0,Trail,Hike,136.0,180.0,1482.0,NaN,NaN,NaN,5mile hike. Was my first ever hike at 3800 met...
6,2026-01-18,0.87,00:10:04,00:08:55,11:31,11:31,3.0,3.0,Road,Easy,156.0,190.0,135.0,NaN,NaN,NaN,Very short run. This was my final run in Mexic...
7,2026-01-21,1.12,00:12:07,00:11:52,10:47,10:47,0.0,0.0,Treadmill,Easy,140.0,151.0,140.0,NaN,NaN,NaN,First official day of training where I will be...
8,2026-01-24,3.04,00:31:02,00:30:15,10:13,10:13,0.0,0.0,Treadmill,Easy,148.0,166.0,383.0,NaN,NaN,NaN,Wanted to run for 30 minutes nonstop at a easy...
9,2026-01-25,1.36,00:13:14,00:13:09,9:42,9:42,3.0,10.0,Road,Easy,163.0,191.0,186.0,NaN,NaN,NaN,Ran to Miles and Keitlin's apartment to watch ...
